<a href="https://colab.research.google.com/github/NataIss/codeforces/blob/main/Codeforces_15_NumOfUsersByContest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import requests
import pandas as pd
import numpy as np
import time

In [2]:
try:  
    data = pd.read_json("https://codeforces.com/api/contest.list?gym=false")
    con = pd.DataFrame.from_records(data['result'])
    con
except (BaseException) as e:    
    print(e)
con

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds
0,1635,Codeforces Round #TBA (Div. 2),CF,BEFORE,False,7200,1645367700,-2603254
1,1634,Codeforces Round #770 (Div. 2),CF,BEFORE,False,7200,1644158100,-1393654
2,1633,Educational Codeforces Round 122 (Rated for Di...,ICPC,BEFORE,False,7200,1643639700,-875254
3,1632,Codeforces Round #769 (Div. 2),CF,BEFORE,False,7200,1643553300,-788854
4,1630,Codeforces Round #768 (Div. 1),CF,BEFORE,False,7200,1643294100,-529654
...,...,...,...,...,...,...,...,...
1551,5,Codeforces Beta Round #5,ICPC,FINISHED,False,7200,1269100800,373663646
1552,4,Codeforces Beta Round #4 (Div. 2 Only),ICPC,FINISHED,False,7200,1268395200,374369246
1553,3,Codeforces Beta Round #3,ICPC,FINISHED,False,7200,1267963200,374801246
1554,2,Codeforces Beta Round #2,ICPC,FINISHED,False,7200,1267117200,375647246


In [3]:
con['d']=pd.to_datetime(con['startTimeSeconds'],unit='s')
con.set_index('d',inplace=True)
con['Y']=con.index.year
con['M']=con.index.month
con['H']=con.index.hour
con['W']=con.index.dayofweek


In [4]:
con=con[con['Y']==2021]
con

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds,Y,M,H,W
d,,,,,,,,,,,,
2021-12-29 15:35:00,1616,Good Bye 2021: 2022 is NEAR,CF,FINISHED,False,7200,1640792100,1972346,2021,12,15,2
2021-12-28 13:35:00,1623,Codeforces Round #763 (Div. 2),CF,FINISHED,False,7200,1640698500,2065946,2021,12,13,1
2021-12-27 14:35:00,1622,Educational Codeforces Round 120 (Rated for Di...,ICPC,FINISHED,False,7200,1640615700,2148746,2021,12,14,0
2021-12-24 14:35:00,1615,Codeforces Global Round 18,CF,FINISHED,False,9000,1640356500,2407946,2021,12,14,4
2021-12-20 14:35:00,1619,Codeforces Round #762 (Div. 3),ICPC,FINISHED,False,8100,1640010900,2753546,2021,12,14,0
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-14 14:35:00,1473,Educational Codeforces Round 102 (Rated for Di...,ICPC,FINISHED,False,7200,1610634900,32129546,2021,1,14,3
2021-01-08 14:35:00,1467,Codeforces Round #695 (Div. 2),CF,FINISHED,False,7200,1610116500,32647946,2021,1,14,4
2021-01-05 14:35:00,1470,Codeforces Round #694 (Div. 1),CF,FINISHED,False,7200,1609857300,32907146,2021,1,14,1


In [5]:
d={}

In [11]:
for j in con['id']: 
    i=0
    while i<3:
        try:
            time.sleep(0.7)
            link='https://codeforces.com/api/contest.standings?contestId='+str(j)+'&showUnofficial=false'
            data = pd.read_json(link)
            if data['status'][0]=='OK':
                dfAll = pd.DataFrame.from_records(data['result']['rows'])
                d[j]=len(dfAll)
                i=i+5
                print(j) + "****"
        except (BaseException) as e:    
                time.sleep(1)
                i=i+1
                #ratedListAll[ratedListAll['handle']==j][0]
                if i==3:
                    print(j)
                    print(e)
d

1616
1623
1622
1615
1619
1620
1617
1618
1585
1591
1608
1597
HTTP Error 400: 
1613
1609
1614
1611
1610
1612
1584
1588
1589
1605
1596
HTTP Error 400: 
1607
1603
1604
1606
1601
1602
1582
1583
1586
1595
HTTP Error 400: 
1593
1598
1599
1600
1576
1594
1571
1592
1575
1570
1578
1580
1581
1579
1574
1572
1573
1566
1569
1567
1556
1562
1558
1561
1563
1560
1559
1557
1548
1549
1555
1554
1552
1551
1553
1544
1530
1550
1545
1546
1547
1543
1542
1533
1540
1541
1532
1539
1537
1534
1538
1536
1535
1523
1526
1524
1528
1529
1531
1527
1522
1525
1521
1520
1515
1519
1517
1516
1514
1508
1509
1511
1513
1512
1510
1503
1504
1505
1498
1506
1483
1484
1482
1499
1497
1500
1501
1495
1496
1488
1493
1494
1489
1491
1492
1486
1490
1487
1485
1479
1480
1481
1476
1477
1478
1475
1474
1473
1467
1470
1471
1472


{1467: 14038,
 1470: 1004,
 1471: 13199,
 1472: 16219,
 1473: 15008,
 1474: 13105,
 1475: 19332,
 1476: 15203,
 1477: 799,
 1478: 15228,
 1479: 1068,
 1480: 14037,
 1481: 14257,
 1482: 779,
 1483: 891,
 1484: 9553,
 1485: 9411,
 1486: 13187,
 1487: 13860,
 1488: 698,
 1489: 1014,
 1490: 12061,
 1491: 10732,
 1492: 9174,
 1493: 9978,
 1494: 11826,
 1495: 1111,
 1496: 9670,
 1497: 12682,
 1498: 10637,
 1499: 11857,
 1500: 703,
 1501: 6879,
 1503: 1040,
 1504: 11315,
 1505: 9903,
 1506: 12808,
 1508: 943,
 1509: 13474,
 1510: 833,
 1511: 13484,
 1512: 15101,
 1513: 10175,
 1514: 11472,
 1515: 13044,
 1516: 12033,
 1517: 11248,
 1519: 14799,
 1520: 18743,
 1521: 13453,
 1522: 517,
 1523: 9010,
 1524: 244,
 1525: 11659,
 1526: 14420,
 1527: 15567,
 1528: 943,
 1529: 16171,
 1530: 11705,
 1531: 992,
 1532: 1719,
 1533: 912,
 1534: 8213,
 1535: 17842,
 1536: 11230,
 1537: 16403,
 1538: 16304,
 1539: 10364,
 1540: 770,
 1541: 15084,
 1542: 14822,
 1543: 13958,
 1544: 192,
 1545: 938,
 1546: 10

In [15]:
num =pd.DataFrame.from_dict(d,orient='index')      

In [16]:
num

,0
1616,14127
1623,10110
1622,15928
1615,12606
1619,15498
...,...
1473,15008
1467,14038
1470,1004
1471,13199


In [24]:
res = pd.merge(con, num, how="inner",left_on='id',right_index=True)
res

,id,name,type,phase,frozen,durationSeconds,startTimeSeconds,relativeTimeSeconds,Y,M,H,W,0
d,,,,,,,,,,,,,
2021-12-29 15:35:00,1616,Good Bye 2021: 2022 is NEAR,CF,FINISHED,False,7200,1640792100,1972346,2021,12,15,2,14127
2021-12-28 13:35:00,1623,Codeforces Round #763 (Div. 2),CF,FINISHED,False,7200,1640698500,2065946,2021,12,13,1,10110
2021-12-27 14:35:00,1622,Educational Codeforces Round 120 (Rated for Di...,ICPC,FINISHED,False,7200,1640615700,2148746,2021,12,14,0,15928
2021-12-24 14:35:00,1615,Codeforces Global Round 18,CF,FINISHED,False,9000,1640356500,2407946,2021,12,14,4,12606
2021-12-20 14:35:00,1619,Codeforces Round #762 (Div. 3),ICPC,FINISHED,False,8100,1640010900,2753546,2021,12,14,0,15498
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-14 14:35:00,1473,Educational Codeforces Round 102 (Rated for Di...,ICPC,FINISHED,False,7200,1610634900,32129546,2021,1,14,3,15008
2021-01-08 14:35:00,1467,Codeforces Round #695 (Div. 2),CF,FINISHED,False,7200,1610116500,32647946,2021,1,14,4,14038
2021-01-05 14:35:00,1470,Codeforces Round #694 (Div. 1),CF,FINISHED,False,7200,1609857300,32907146,2021,1,14,1,1004


In [25]:
res['date']=res.index
res.set_index('id', inplace=True)

In [26]:
res.rename(columns={0: 'NumOfUsers'},inplace=True)

In [27]:
res1=res[['name','type','Y','M','H','NumOfUsers','date']]
res1.head()

,name,type,Y,M,H,NumOfUsers,date
id,,,,,,,
1616,Good Bye 2021: 2022 is NEAR,CF,2021,12,15,14127,2021-12-29 15:35:00
1623,Codeforces Round #763 (Div. 2),CF,2021,12,13,10110,2021-12-28 13:35:00
1622,Educational Codeforces Round 120 (Rated for Di...,ICPC,2021,12,14,15928,2021-12-27 14:35:00
1615,Codeforces Global Round 18,CF,2021,12,14,12606,2021-12-24 14:35:00
1619,Codeforces Round #762 (Div. 3),ICPC,2021,12,14,15498,2021-12-20 14:35:00


In [22]:
res1.to_excel('NumOfUsersByContestIn2021.xlsx')

In [23]:
res1

,name,type,Y,M,H,NumOfUsers,date
id,,,,,,,
1616,Good Bye 2021: 2022 is NEAR,CF,2021,12,15,14127,1616
1623,Codeforces Round #763 (Div. 2),CF,2021,12,13,10110,1623
1622,Educational Codeforces Round 120 (Rated for Di...,ICPC,2021,12,14,15928,1622
1615,Codeforces Global Round 18,CF,2021,12,14,12606,1615
1619,Codeforces Round #762 (Div. 3),ICPC,2021,12,14,15498,1619
...,...,...,...,...,...,...,...
1473,Educational Codeforces Round 102 (Rated for Di...,ICPC,2021,1,14,15008,1473
1467,Codeforces Round #695 (Div. 2),CF,2021,1,14,14038,1467
1470,Codeforces Round #694 (Div. 1),CF,2021,1,14,1004,1470
